In [1]:
import boto3
from IPython.display import clear_output, Image, display, HTML
import numpy as np
import cv2
import base64
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import time
import json
output_notebook()

Loading BokehJS ...

In [2]:
STREAM_NAME = "pi4-001"
kvs = boto3.client("kinesisvideo")
# Grab the endpoint from GetDataEndpoint
endpoint = kvs.get_data_endpoint(
    APIName="GET_HLS_STREAMING_SESSION_URL",
    StreamName=STREAM_NAME
)['DataEndpoint']
print(endpoint)

https://b-647daf39.kinesisvideo.us-west-2.amazonaws.com


In [3]:
from object_detection import ObjectDetection
detector = ObjectDetection()

Loaded from params file:  object_detector_epoch200_11_08_2019_21_56_15.params


In [4]:
class VideoPlayer(object):
    def __init__(self):
        self._init = False
        self._myImage = None
        
    def __call__(self, frame):
        if frame is None:
            return
        if self._init is False:
            self.init_display(frame)
            self._init = True
        else:
            self.update_display(frame)

    def init_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) # because Bokeh expects a RGBA image
        frame=cv2.flip(frame, 0) # because Bokeh flips vertically
        width=frame.shape[1]
        height=frame.shape[0]
        p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=width, height=height)
        self._myImage = p.image_rgba(image=[frame], x=0, y=0, dw=width, dh=height)
        show(p, notebook_handle=True)
    
    def update_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        frame=cv2.flip(frame, 0) 
        self._myImage.data_source.data['image']=[frame]
        push_notebook()

In [5]:
from collections import Counter


class NaiveMonitor(object):
    def __init__(self, classes, thresh=0.5, hand_activate_frames=2, hand_deactivate_frames=60):
        self.hand_cnt = 0
        self.no_hand_cnt = 0
        self.start_trans = False
        self.end_trans = False
        self.in_trans = False
        self.curr_item_cnt = Counter()
#         self.max_item_cnt = [4, 1, 1]
        self.pre_msg = ''
        self.pre_msg2 = ''
        self.classes = classes
        self.thresh = thresh
        self.hand_index = self.classes.index('hand')
        self.hand_activate_frames = hand_activate_frames
        self.hand_deactivate_frames = hand_deactivate_frames
        
    def update(self, frame, class_IDs, scores, bounding_boxes):
        hand = False
        item_cnt = Counter()
        for i in range(len(class_IDs)):
            if scores[i] <= self.thresh:
                continue
            cid = int(class_IDs[i])
            score = float(scores[i])
            bbox = bounding_boxes[i]
            xmin, ymin, xmax, ymax = [int(x) for x in bbox]
            # draw bounding box
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color=(0, 255, 0), thickness=1)
            cv2.putText(frame, str(self.classes[cid])+':'+str(round(score, 2)), (xmin+10, ymin+10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, thickness=1, color=(255, 255, 255))
            
            if cid == self.hand_index:
                hand = True
            else:
                # update counter
                item_cnt.update({self.classes[cid]: 1})
                
        if not self.curr_item_cnt:
            self.curr_item_cnt = item_cnt
        if not hand:
            self.no_hand_cnt += 1
        elif not self.in_trans:
            self.hand_cnt += 1
        else:
            # reset no hand cnt when transaction is ongoing
            self.no_hand_cnt = 0

        # switch transaction stage
        if self.hand_cnt >= self.hand_activate_frames:
            self.start_trans = True
            self.end_trans = False
            self.hand_cnt = 0
            self.no_hand_cnt = 0
        elif self.no_hand_cnt >= self.hand_deactivate_frames:
            self.start_trans = False
            self.end_trans = True
            self.hand_cnt = 0
            self.no_hand_cnt = 0

        msg = ''
        if self.start_trans:
            msg = 'Start Transaction'
            self.start_trans = False
            self.in_trans = True
        elif self.end_trans:
            if self.in_trans: 
                self.in_trans = False
                msg = 'End Transaction'
                for i in item_cnt:
                    diff_item = item_cnt[i] - self.curr_item_cnt[i]
                    if diff_item != 0:
                        char = ''
                        if diff_item > 0:
                            char = '+'
                        msg += ' ' + i +': ' + char + str(diff_item)
                self.curr_item_cnt = item_cnt
            self.end_trans = False
        else:
            msg = self.pre_msg

        msg2 = ''
        for i in sorted(item_cnt.keys()):
            msg2 += i +': ' + str(item_cnt[i]) + ' '
        msg2 += '[<-] '
        for i in sorted(self.curr_item_cnt.keys()):
            msg2 += i +': ' + str(self.curr_item_cnt[i]) + ' '

        cv2.putText(frame, msg, (40, 40), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, thickness=2, color=(255, 255, 255))
        cv2.putText(frame, msg2, (40, 60), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, thickness=1, color=(255, 255, 255))
        self.pre_msg = msg
        self.pre_msg2 = msg2
        return frame

In [6]:
# Grab the HLS Stream URL from the endpoint
kvam = boto3.client("kinesis-video-archived-media", endpoint_url=endpoint)
url = kvam.get_hls_streaming_session_url(
    StreamName=STREAM_NAME,
    PlaybackMode="LIVE"
)['HLSStreamingSessionURL']
vcap = cv2.VideoCapture(url)
#vcap.set(cv2.CAP_PROP_BUFFERSIZE, 3)

player = VideoPlayer()
monitor = NaiveMonitor(detector.classes, hand_deactivate_frames=60)

while(True):
    # Capture frame-by-frame
    read_start = time.time()
    ret, frame = vcap.read()
    read_end = time.time()
    # print('read time:', read_end-read_start)

    if frame is not None:
        start = time.time()
        frame = cv2.flip(frame, -1)
        
        # use GluonCV
        detect_start = time.time()
        class_IDs, scores, bounding_boxes = detector.detect_image(frame)
        detect_end = time.time()
        # print('detect time:', detect_end-detect_start)
        
        frame = monitor.update(frame, class_IDs[0], scores[0], bounding_boxes[0])
        
        # Display the resulting frame
        player(frame)
        end = time.time()
        #print('all time:', end-start)
    else:
        print("Frame is None")
        break

# When everything done, release the capture
vcap.release()
print("Video stop")

KeyboardInterrupt: 